# Name : Ko Tsuruta
...

#Title : VERTIGO
...

#Idea / Concept / Inspiration : Inspired by Roman Verostko
                                The drawing expresses not only a colorful labyrinth, 
                                but also the feeling when we enter the labyrinth.
...

#Method : Step 1--Draw 2 NURBS crv. using random.
                  1st crv. is biased to go along X-axis, 
                  2nd crv. is biased to go along Y-axis.
          Step 2--Prepare a perlin noise to change the color of crv. gradationally.
          Step 3--Array 1st crv. along 2nd crv.
...

In [3]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import random
import noise
import math

from ipywidgets import interact, widgets
from scipy import interpolate
from geomdl import NURBS

In [4]:
class Dizzy():
    
    def __init__(self, res):
        self.res = res
        self.out = 0
        self.out2 = 0
        self.col = []
    
    def crv(self,crv_type: str, scale, rangex, rangey, iteration):
        ctrlx = []
        ctrly = []
        
        if crv_type == 'crv1':
            for i in range(iteration):
                ctrlx.append(i + rangex)
                ctrly.append(random.randint(0,rangey))
        if crv_type == 'crv2':
            for i in range(iteration):
                ctrly.append(i + rangey)
                ctrlx.append(random.randint(0,rangex))

        x = ctrlx
        y = ctrly
        l=len(x)  

        t=np.linspace(0,1,l-2,endpoint=True)
        t=np.append([0,0,0],t)
        t=np.append(t,[1,1,1])

        tck=[t,[x,y],3]
        u3=np.linspace(0,1,(max(l*2,self.res)),endpoint=True)
        
        self.out = interpolate.splev(u3,tck)
        #print(self.out)
        return self.out
    
    def pnoise(self, scale, octaves, persistence, lacunarity):
        pn = []
        
        for i in range(self.res):
            r = noise.pnoise2(i/scale, 
                            i/scale, 
                            octaves=octaves, 
                            persistence=persistence, 
                            lacunarity=lacunarity, 
                            repeatx=self.res, 
                            base=0)
            r_remapped = 0.5*(r+1)
            pn.append(r_remapped)
        return pn
    
    
    def array(self, res, iteration1, iteration2, linewidth):
        plt.figure(figsize=(30,30))

        d = Dizzy( res )
        c1 = d.crv('crv1', 10, 10, 5, iteration1)
        c2 = d.crv('crv2', 10, 5, 5, iteration2)
        pn1 = d.pnoise(10, 6, 0.1, 1.0)
        pn2 = d.pnoise(20, 6, 0.5, 1.0)
        lw = linewidth

        col = []

        for i in range(len(c1[0])):
            col.append([pn1[i], pn2[i], 1])
            #print(col)
            _x = c1[0][i]-c2[0][i]
            _y = c1[1][i]-c2[1][i]
            plt.plot(c1[0]-_x, c1[1]-_y, color = col[i], linewidth=lw)

In [5]:
@interact(res = widgets.IntSlider(min=100, max=3000, step=100, value=1000),
         iteration1 = widgets.IntSlider(min=20, max=100, step=1, value=30),
         iteration2 = widgets.IntSlider(min=20, max=100, step=1, value=75),
         linewidth = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0))

def dizzy(res, iteration1, iteration2, linewidth):
    d = Dizzy( res )
    d.array(res, iteration1, iteration2, linewidth)
    
    plt.figure(figsize=(30,30))
    plt.axis("off")
    plt.show()

interactive(children=(IntSlider(value=1000, description='res', max=3000, min=100, step=100), IntSlider(value=3…